In [11]:
using JuMP, Gurobi, Random, Distributions, LinearAlgebra, Plots, CSV, DataFrames, Parsers


# Lecture 7 Notes

## Nominal Model - Known returns

### Parameters

In [15]:
# Total number of assets; Number of risky assets = M-1
M = 20 

# Number of trading periods
N = 5

# brokerage fees
csell = 0.03
cbuy = 0.02

# returns by asset and period
#  riskless asset
r₀ = ones(N) 
rₘ = rand(N,M)

5×20 Array{Float64,2}:
 0.418353  0.579713   0.969058  0.857209  …  0.440889  0.0687617  0.292623
 0.998458  0.742244   0.803985  0.799554     0.566242  0.316321   0.981787
 0.481592  0.0593575  0.42382   0.841182     0.755878  0.475002   0.27427
 0.188815  0.242246   0.41115   0.541593     0.902843  0.592553   0.516749
 0.880004  0.0602065  0.803132  0.187251     0.050698  0.165822   0.814617

### Formulation

In [13]:
#mod = Model(solver = GurobiSolver())
mod = Model(Gurobi.Optimizer)

##### Decision Variables #####

# x, number of dollar holdings at the beginning of time period t on asset m
@variable(mod, x[1:N, 0:M] >= 0)

# u, dollar sells on asset m at time t
@variable(mod, u[1:N, 0:M] >= 0)

# v, dollar buys on asset m at time t
@variable(mod, v[1:N, 0:M] >= 0)

##### Constraints #####

# buy and sell constraints
@constraint(mod, [t=2:N,m=1:M], x[t,m] == (1+rₘ[t-1,m])*(x[t-1,m] - u[t-1,m] + v[t-1,m]))
@constraint(mod, [t=2:N], x[t,0] == (1+r₀[t-1])*(x[t-1,0] + sum((1-csell)*u[t-1,m] for m = 1:M) - sum((1-cbuy)*v[t-1,m] for m = 1:M)))

## NEED A CONSTRAINT ON THE AMOUNT OF CASH AVAILABLE AT T=0
@constraint(mod, sum(x[1,m] for m in 0:M) <= 1)

##### Objective Function #####

# maximize expected final wealth
@objective(mod, Max, sum(x[N,m] for m in 0:M))


#solve(mod)
optimize!(mod);

Academic license - for non-commercial use only - expires 2021-07-08
Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 85 rows, 315 columns and 509 nonzeros
Model fingerprint: 0x7d1cea4d
Coefficient statistics:
  Matrix range     [1e+00, 2e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+00]
Presolve removed 21 rows and 131 columns
Presolve time: 0.00s
Presolved: 64 rows, 184 columns, 367 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    4.0000000e+30   5.000000e+29   4.000000e+00      0s
      63    1.6000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 63 iterations and 0.00 seconds
Optimal objective  1.600000000e+01

User-callback calls 107, time in user-callback 0.00 sec


In [14]:
getvalue.(x)

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, Base.OneTo(5)
    Dimension 2, 0:20
And data, a 5×21 Array{Float64,2}:
  1.0  0.0  0.0  0.0  0.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
  2.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  4.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
  8.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 16.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0

### Rolling optimization

## Robust Model - Reformulation

### Parameters

In [55]:
# Cumulative Returns for at time 0
R₀ = ones(M)

#Cumulative Returns at time t for asset m
Rₜ = []
for t in 1:N:
    
        


4-element Array{Float64,1}:
 1.0
 1.0
 1.0
 1.0

In [ ]:
rmod = Model(solver = GurobiSolver())

##### Decision Variables #####

# x, number of dollar holdings at the beginning of time period t on asset m
@variable(rmod, x[1:N, 0:M] >= 0)

# u, dollar sells on asset m at time t
@variable(rmod, u[1:N, 0:M] >= 0)

# v, dollar buys on asset m at time t
@variable(rmod, v[1:N, 0:M] >= 0)

# Reformulation Variables


##### Constraints #####

# buy and sell constraints
@constraint(rmod, [t=2:N,m=1:M], x[t,m] == (1+rₘ[t-1,m])*(x[t-1,m] - u[t-1,m] + v[t-1,m]))
@constraint(rmod, [t=2:N], x[t,0] == (1+r₀[t-1])*(x[t-1,0] + sum((1-csell)*u[t-1,m] for m = 1:M) - sum((1-cbuy)*v[t-1,m] for m = 1:M)))

########### NEED A CONSTRAINT ON THE AMOUNT OF CASH AVAILABLE AT T=0
@constraint(rmod, sum(x[1,m] for m in 0:M) <= 1)




##### Objective Function #####

# maximize expected final wealth
@objective(rmod, Max, sum(x[N,m] for m in 0:M))

solve(rmod)